# Monte Carlo Tree Search 

> Fill in a module description here

In [ ]:
#| default_exp mcts

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

- Step 1: It takes the current game state
- Step 2: It runs multiple random game simulations starting from this game state
- Step 3: For each simulation, the final state is evaluated by a score
- Step 4: It only remembers the next move of each simulation and accumulates the score for that move
- Step 5: After the simulation is done, it returns the next move with the highest score

In [ ]:
import random
import ast